In [1]:
import numpy as np
#import scipy
import math
from scipy import integrate, LowLevelCallable
from scipy.integrate import odeint,solve_ivp
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
import os, ctypes



In [2]:
#% spatial grid
L = 0.5                                             #; % reactor length (m)
z01 = 0.0                                           #;
zL1 = 0.14                                          # ; % entrance (inert) section
z02 = zL1                                           #;
zL2 = z02 + 0.095                                   #; % catalyst section
z03 = zL2                                           #;
zL3 = L                                             #; %exit (inert) section
n1 = 71                                             #;
n2 = 71                                             #;
n3 = 71                                             #;
dz1 = (zL1-z01) / (n1-1)                          #;
dz2 = (zL2-z02) / (n2-1)                          #;
dz3 = (zL3-z03) / (n3-1)                          #;
v   = 0.203                                       # intersticial velocity, m/s
num_x = 0.00001                                   # a small added to separate reactor zones

z1 = np.arange(z01,zL1+num_x,dz1,dtype=float)[:, np.newaxis]                       #' ;
z2 = np.arange(z02,zL2+num_x,dz2,dtype=float)[:, np.newaxis]                       #' ;
z3 = np.arange(z03,zL3+num_x,dz3,dtype=float)[:, np.newaxis]                       #' ;
z  = np.concatenate((z1,z2,z3), axis=0)

In [3]:
def weights(zd,zs,ns,m):
        
    # input Parameters
    
    # zd, location where the derivative is to be computed
    # ns, number of points in the stencil    
    # zs(ns), stencil of the finite difference scheme
    # m, highest derivative for which weights are sought
    # output Parameter
    # w(1:ns,1:m+1), weights at grid locations z(1:ns) for derivatives
    # of order 0:m, found in w(1:ns,1:m+1)
    
    c1 = 1.0
    c4 = zs[0]-zd
    
    w = np.zeros((ns,m+1))
    w[0,0] = 1.0
    
    for i in range(1,ns):
        mn = min(i,m)
        c2 = 1.0
        c5 = c4
        c4 = zs[i]-zd
        
        for j in range(i):
            c3 = zs[i]-zs[j]
            c2 = c2*c3
            
            if j==i-1:
                for k in range(mn,0,-1):
                    w[i,k] = c1*(k*w[i-1,k-1] - c5*w[i-1,k])/c2
                w[i,0] = -c1*c5*w[i-1,0]/c2
                
            for k in range(mn,0,-1):
                w[j,k] = (c4*w[j,k] - k*w[j,k-1])/c3
            w[j,0] = c4*w[j,0]/c3
        c1 = c2
    
    return w

In [4]:
# m = 2
# ns = 5
# zs = z[0:ns]
# #print(zs)

# for i in range(0,3):
#     zd = z[i]

# c1 = 1.0
# c4 = zs[0]-zd
# # zs.shape

In [5]:
# m=1      
# ns=5
    
# #sparse discretization matrix 
# n = len(z)
# #D = sparse(n,n), from matlab
# D = csc_matrix((n,n), dtype=int).toarray()
    
# #(1)  finite difference approximation for positive v
    
# if (v > 0):
#     #boundary conditions
#     zs = z[0:ns]
#     for i in range(0,3):
#         zd = z[i]
#         w = weights(zd,zs,ns,m)
#         D[i,0:ns] = w[0:ns,m].T
            
# #         # interior points
#         for i in range(3,n-1):
#             zs = z[i-3:i+2]
#             zd = z[i]
#             w  = weights(zd,zs,ns,m)
#             D[i,i-3:i+2]=w[0:ns,m]# m is calculate
                
#         #boundary point
#         zs = z[n-ns:n]
#         zd = z[n-1]
#         w  = weights(zd,zs,ns,m)
#         D[n-1,n-ns:n] = w[0:ns,m]
            
#     #(2)  finite difference approximation for negative v
#     if (v < 0):
#         # boundary point
#         zs = z[0:ns]
#         zd = z[0]
#         w  = weights(zd,zs,ns,m)
#         D[0,0:ns] = w[0:ns,m] # these need to transposed
        
#         # interior points
#         for i in range(1,n-2):            
#             zs = z[i-1:i+4]
#             zd = z[i]
#             w  = weights(zd,zs,ns,m)
#             D[i,i-1:i+4] = w[0:ns,m] # can be checked later
            
#         # boundary points
#         zs = z[n-ns:n] # Check later the indices
#         for i in range(3,0,-1):
#             zd = z[n-i]
#             w  = weights(zd,zs,ns,m)
#             D[n-i,n-5:n]= w[0:ns,m]

In [6]:
def five_point_biased_upwind_D1(z,v):
    
    m=1
    ns=5
    #sparse discretization matrix
    
    n = len(z)
    #D = sparse(n,n), from matlab
    D = csc_matrix((n,n), dtype=int).toarray()
    #(1)  finite difference approximation for positive v
    
    if (v > 0):
        #boundary conditions
        zs = z[0:ns]
        for i in range(0,3):
            zd = z[i]
            w = weights(zd,zs,ns,m)
            D[i,0:ns] = w[0:ns,m] #.T
            
        # interior points
        for i in range(3,n-1):
            zs = z[i-3:i+2]
            zd = z[i]
            w  = weights(zd,zs,ns,m)
            D[i,i-3:i+2]=w[0:ns,m]# m is calculate
            
        #boundary point
        zs = z[n-ns:n]
        zd = z[n-1]
        w  = weights(zd,zs,ns,m)
        D[n-1,n-ns:n] = w[0:ns,m]
            
    #(2)  finite difference approximation for negative v
    if (v < 0):
        # boundary point
        zs = z[0:ns]
        zd = z[0]
        w  = weights(zd,zs,ns,m)
        D[0,0:ns] = w[0:ns,m] # these need to transposed
        
        # interior points
        for i in range(1,n-2):            
            zs = z[i-1:i+4]
            zd = z[i]
            w  = weights(zd,zs,ns,m)
            D[i,i-1:i+4] = w[0:ns,m] # can be checked later
            
        # boundary points
        zs = z[n-ns:n] # Check later the indices
        for i in range(3,0,-1):
            zd = z[n-i]
            w  = weights(zd,zs,ns,m)
            D[n-i,n-5:n]= w[0:ns,m]            
    return D

In [7]:
D = five_point_biased_upwind_D1(z,v)

C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_

In [12]:
# def five_point_biased_upwind_D1(z,v):
    
#     m=1      
#     ns=5
    
#     #sparse discretization matrix 
#     n = len(z)
#     #D = sparse(n,n), from matlab
#     D = csc_matrix((n,n), dtype=int).toarray()
    
#     #(1)  finite difference approximation for positive v
    
#     if (v > 0):
#         #boundary conditions
#         zs = z[0:ns]
#         for i in range(0,3):
#             zd = z[i]
#             w = weights(zd,zs,ns,m)
#             D[i,0:ns] = w[0:ns,m]
            
#         # interior points
#         for i in range(3,n):
#             zs = z[i-3:i+1]
#             zd = z[i]
#             w  = weights(zd,zs,ns,m)
#             D[i,i-3:i+1]=w[1:ns,m+1]
                
#         #boundary point
#         zs = z[n-ns+1:n]
#         zd = z[n]
#         w  = weights(zd,zs,ns,m)
#         D[n,n-ns+1:n] = w[1:ns,m+1]
            
#     #(2)  finite difference approximation for negative v
#     if (v < 0):
#         # boundary point
#         zs = z[0:ns]
#         zd = z[0]
#         w  = weights(zd,zs,ns,m)
#         D[0,0:ns]= w[1:ns,m+1] # these need to transposed
        
#         # interior points
#         for i in range(1,n-3):
#             zs = z[i-1:i+3]
#             zd = z[i]
#             w  = weights(zd,zs,ns,m)
#             D[i,i-1:i+3] = w[0:ns,m+1]
            
#         # boundary points
#         zs = z[n-ns+1:n] # Check later the indices
#         for i in range(2,-1,-1):
#             zd = z(n-i+1)
#             w  = weights(zd,zs,ns,m)
#             D[n-i+1,n-4:n]= w[0:ns,m+1]
#     return D
            

In [46]:

m=2                # highest derivative for which weights are sought
ns=5               # number of points in the stencil
    
# sparse discretization matrix
    
n = len(z)
#D = sparse(n,n)
D = csc_matrix((n,n), dtype=int).toarray()
    
# boundary points
    
zs = z[0:ns]
    
for i in range(0,2):
    zd = z[i]
    # print(zd)
    w = weights(zd,zs,ns,m)
    #print(w)
    D[i,0:ns] = w[0:ns,m] # need to check the orientation whether transpose is needed
    #print(D)
    
    # interior points
for i in range(2,n-2):
    zs = z[i-2:i+3]
    zd = z[i]
    w  = weights(zd,zs,ns,m)
    D[i,i-2:i+3]=w[0:ns,m] # check this later
    
    # boundary points 
zs = z[n-ns:n]
for i in range(2,0,-1): 
    zd = z[n-i]
    w  =  weights(zd,zs,ns,m)
    D[n-i,n-5:n]=w[0:ns,m] # check the return matrix
    

C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in subtract
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply
C:\Users\kabugoj1\AppData\Local\conda\conda\envs\my_root

In [47]:
D

array([[  729166, -2166666,  2375000, ...,        0,        0,        0],
       [  229166,  -416666,   124999, ...,        0,        0,        0],
       [  -20833,   333333,  -625000, ...,        0,        0,        0],
       ...,
       [       0,        0,        0, ...,  -174439,    93034,    -5814],
       [       0,        0,        0, ...,    34887,  -116292,    63961],
       [       0,        0,        0, ...,   662869,  -604722,   203512]])

34887

In [9]:
def five_point_centered_D2(z):
    
    # parameters
    m=2                # highest derivative for which weights are sought
    ns=5               # number of points in the stencil
    
    # sparse discretization matrix
    
    n = len(z)
    #D = sparse(n,n)
    D = csc_matrix((n,n), dtype=int).toarray()
    
    # boundary points
    
    zs = z[0:ns]
    
    for i in range(0,2):
        zd = z[i]
        w = weights(zd,zs,ns,m)
        D[i,0:ns] = w[0:ns,m] # need to check the orientation whether transpose is needed
        
        
    # interior points
    for i in range(2,n-2):
        zs = z[i-2:i+3]
        zd = z[i]
        w  = weights(zd,zs,ns,m)
        D[i,i-2:i+3]=w[0:ns,m] # check this later
    
    
    # boundary points 
    zs = z[n-ns:n]
    
    for i in range(2,0,-1): 
        zd = z[n-i]
        w  =  weights(zd,zs,ns,m)
        D[n-i,n-5:n]=w[0:ns,m] # check the return matrix
    
    return D
        
        